## Introduction

This code was used to process geospatial raster data from Planet to be useable for this project.

The data was stored in GCS Buckets and processed as a NumPy array to ensure:
- Color Normalization
- Consistent Band Order
- Consustent Image Bounds for each region
- LZW Compression
- Consistent Naming Convention

Then, each image was again saved as a raster tiff, back in a GCS bucket.

## Setup

Note: to avoid package discrepencies, use a coda environment, and add it to jupyter by following instructions on this link: https://medium.com/@nrk25693/how-to-add-your-conda-environment-to-your-jupyter-notebook-in-just-4-steps-abeab8b8d084

Use the following command creating the conda env (use the packages as part of the creation of the env):

conda create -n [name of virtual env] -c conda-forge rasterio jupyter geojson matplotlib

In [ ]:
## Launch jupyter through the command line instead of opening manually

## Use this to check what env jupyter is reading from:
# !conda info

## To delete a kernel:
# jupyter kernelspec list
# jupyter kernelspec uninstall unwanted-kernel

In [ ]:
import rasterio
from rasterio.warp import transform_bounds
from rasterio.features import bounds as calculate_bounds
import matplotlib.pyplot as plt
from rasterio.crs import CRS
from rasterio.windows import Window
from google.cloud import storage
#from osgeo import gdal
#import gdal
#from osgeo import osr
import time
import numpy as np
#import xarray as xr
import os
#import asyncio
#import aiofiles
#import aiohttp
import geojson
from PIL import Image
import concurrent.futures
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='/Users/mansi/Downloads/planet-devrel-prod-04da32672001.json'

In [ ]:
print('Landsat on Google:')
filepath = 'https://storage.googleapis.com/gcp-public-data-landsat/LC08/01/042/034/LC08_L1TP_042034_20170616_20170629_01_T1/LC08_L1TP_042034_20170616_20170629_01_T1_B4.TIF'
with rasterio.open(filepath) as src:
    print(src.profile)

Landsat on Google:
{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 7821, 'height': 7951, 'count': 1, 'crs': CRS.from_wkt('PROJCS["WGS 84 / UTM zone 11N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-117],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32611"]]'), 'transform': Affine(30.0, 0.0, 204285.0,
       0.0, -30.0, 4268115.0), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}


### Code for Example

In [ ]:
# Note: change url to 87 (CentralPark) or 88 (Everything Else)
gs_access_key= "gs_access_key"
gs_secret_access_key = "gs_secret_access_key"
url_88 = 'gs://dxd_project_2022/UTM-24000/18N/24E-188N/PF-SR/2018-01-01.tif'
url_87 = 'gs://dxd_project_2022/UTM-24000/18N/24E-187N/PF-SR/2018-01-01.tif'


with rasterio.Env(GS_SECRET_ACCESS_KEY=gs_secret_access_key, GS_ACCESS_KEY_ID=gs_access_key):
    with rasterio.open(url_87) as src:

        print(src.profile)
        print(src.shape)

In [ ]:
with rasterio.Env(GS_SECRET_ACCESS_KEY=gs_secret_access_key, GS_ACCESS_KEY_ID=gs_access_key):
        with rasterio.open(url_87) as src:
            out_meta = src.meta
            print(out_meta)

            # Update the metadata (photometric = RGB)
            # out_meta.update({"driver": "GTiff",
                 # "photometric": "RGB", # need to specify this for TIFF images
                 # "height": src.shape[0],
                 # "width": src.shape[1]})
            # print(out_meta)
        # Read the grid values into numpy arrays
            bands = src.read([1,2,3], window=ProspectPark)
            rgb = np.dstack((bands[2], bands[1], bands[0]))
            # red = src.read(3, window=Window1)
            # green = src.read(2, window=Window1)
            # blue = src.read(1, window=Window1)

        # Function to normalize the grid values
        def normalize(array):
            # Normalizes numpy arrays into scale 0.0 - 1.0
            array_min, array_max = array.min(), array.max()
            return ((array - array_min)/(array_max - array_min))

        # Normalize the bands
        redn = normalize(red)
        greenn = normalize(green)
        bluen = normalize(blue)

        print("Normalized bands")
        print(redn.min(), '-', redn.max(), 'mean:', redn.mean())
        print(greenn.min(), '-', greenn.max(), 'mean:', greenn.mean())
        print(bluen.min(), '-', bluen.max(), 'mean:', bluen.mean())

In [ ]:
# Create RGB natural color composite
rgb = np.dstack((bands[2], bands[1], bands[0]))

# Let's see how our color composite looks like
plt.figure(figsize = (10,12.5))
plt.imshow(rgb)
rgb.shape
src.shape

In [ ]:
# Double check pixel dimensions!! Should be 1920 x 1080
rgb.shape

In [ ]:
# Register GDAL format drivers and configuration options with a
# context manager.
with rasterio.Env(GS_SECRET_ACCESS_KEY=gs_secret_access_key, GS_ACCESS_KEY_ID=gs_access_key):
    with rasterio.open(url_87) as src:
        profile = src.profile.copy()
        print(np.moveaxis(rgb.astype(rasterio.int16), [0, 1, 2], [2, 1, 0]).shape) #210 to 201 for GW # nope jk
        profile.update(
            dtype = rasterio.int16,
            count= 3,
            compress='lzw',
            photometric="RGB",
            height = rgb.shape[0],
            width = rgb.shape[1],
            transform=rasterio.windows.transform(ProspectPark, src.transform))

        with rasterio.open('/Users/mansi/Downloads/example.tif', 'w', **profile) as dst:
            dst.write(np.moveaxis(rgb.astype(rasterio.int16), [0, 1, 2], [2, 1, 0]))

    # At the end of the ``with rasterio.Env()`` block, context
    # manager exits and all drivers are de-registered.

In [ ]:
# Write an array as a raster
# For the new file's profile, we start with the profile of the source
profile = src.profile.copy()

#  Set dtype to int16, and specify LZW compression.
profile.update(
    driver = "GTiff",
    dtype = rasterio.int16,
    count = 3,
    compress ='lzw',
    transform =rasterio.windows.transform(ProspectPark, src.transform),
    height = rgb.shape[0],
    width = rgb.shape[1],
    photometric ="RGB")

    with rasterio.open('/Users/mansi/Downloads/example.tif', 'w', **profile) as dst:
        dst.write(np.moveaxis(rgb.astype(rasterio.int16), [0, 1, 2], [2, 1, 0]))

## Code for Final

Note: change the url below depending on if you're using Central Park (87) or anything else (88) as your AOI:

In [ ]:
gs_access_key= "gs_access_key"
gs_secret_access_key = "gs_secret_access_key"
url_88 = 'gs://dxd_project_2022/UTM-24000/18N/24E-188N/PF-SR/2018-01-01.tif'
url_87 = 'gs://dxd_project_2022/UTM-24000/18N/24E-187N/PF-SR/2018-01-01.tif'


with rasterio.Env(GS_SECRET_ACCESS_KEY=gs_secret_access_key, GS_ACCESS_KEY_ID=gs_access_key):
    with rasterio.open(url_87) as src:

        print(src.profile)
        print(src.shape)

{'driver': 'GTiff', 'dtype': 'int16', 'nodata': None, 'width': 8000, 'height': 8000, 'count': 4, 'crs': CRS.from_epsg(32618), 'transform': Affine(3.0, 0.0, 576000.0,
       0.0, -3.0, 4512000.0), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}
(8000, 8000)


### Central Park

In [ ]:
geojson_cp ={
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -73.99103164672852,
              40.76598152771282
            ],
            [
              -73.96940231323242,
              40.75596974281819
            ],
            [
              -73.93524169921875,
              40.801465703863364
            ],
            [
              -73.95772933959961,
              40.811340708909874
            ],
            [
              -73.99103164672852,
              40.76598152771282
            ]
          ]
        ]
      }
    }

In [ ]:
geojson_bounds_cp = geojson_cp.get('bbox', calculate_bounds(geojson_cp))
geojson_bounds_cp
bounds_cp = transform_bounds(CRS.from_epsg(4326), src.crs, *geojson_bounds_cp)
left, bottom, right, top = bounds_cp
CentralPark = rasterio.windows.from_bounds(left, bottom, right, top, src.transform).round_offsets()

ERROR 1: PROJ: proj_create_from_database: /opt/anaconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


CRSError: The EPSG code is unknown. PROJ: proj_create_from_database: /opt/anaconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.

In [ ]:
# Use 88
def save_and_process_subsets_CP(file_name):
    with rasterio.open(file_name) as src:
        out_meta = src.meta

        # blue green red
        bands = src.read([1,2,3], window=CentralPark)

        # Create RGB natural color composite
        rgb = np.dstack((bands[2], bands[1], bands[0]))

        profile = src.profile.copy()

        #  Set dtype to int16, and specify LZW compression.
        profile.update(
            driver = "GTiff",
            dtype = rasterio.int16,
            count = 3,
            compress ='lzw',
            height = rgb.shape[0],
            width = rgb.shape[1],
            transform =rasterio.windows.transform(CentralPark, src.transform),
            photometric ="RGB")


        with rasterio.open('/Users/mansi/Downloads/CentralPark/' + file_name.split('/')[-1] , 'w', **profile) as dst:
            dst.write(np.moveaxis(rgb.astype(rasterio.int16), [0, 1, 2], [2, 1, 0]))

        print("done ", file_name.split('/')[-1])

    return


### Green-Wood Cemetery

In [ ]:
geojson_gwc ={
    "type": "Feature",
    "properties": {},
    "geometry": {
    "type": "Polygon",
    "coordinates": [
          [
            [
              -73.99446487426758,
              40.66918118282895
            ],
            [
              -74.03360366821288,
              40.64378684722198
            ],
            [
              -74.00699615478514,
              40.62268266420308
            ],
            [
              -73.9654541015625,
              40.64990841734959
            ],
            [
              -73.99446487426758,
              40.66918118282895
            ]

          ]
    ]
    }
}

In [ ]:
geojson_bounds_gwc = geojson_gwc.get('bbox', calculate_bounds(geojson_gwc))
geojson_bounds_gwc
bounds_gwc = transform_bounds(CRS.from_epsg(4326), src.crs, *geojson_bounds_gwc)
left, bottom, right, top = bounds_gwc
GreenWoodCemetery = rasterio.windows.from_bounds(left, bottom, right, top, src.transform).round_offsets()

In [ ]:
# Use 87
def save_and_process_subsets_GWC(file_name):
    with rasterio.open(file_name) as src:
        out_meta = src.meta

        # blue green red
        bands = src.read([1,2,3], window=GreenWoodCemetery)

        # Create RGB natural color composite
        rgb = np.dstack((bands[2], bands[1], bands[0]))

        profile = src.profile.copy()

        #  Set dtype to int16, and specify LZW compression.
        profile.update(
            driver = "GTiff",
            dtype = rasterio.int16,
            count = 3,
            compress ='lzw',
            height = rgb.shape[0],
            width = rgb.shape[1],
            transform =rasterio.windows.transform(GreenWoodCemetery, src.transform),
            photometric ="RGB")

        with rasterio.open('/Users/mansi/Downloads/GreenWoodCemetery/' + file_name.split('/')[-1] , 'w', **profile) as dst:
            dst.write(np.moveaxis(rgb.astype(rasterio.int16), [0, 1, 2], [2, 1, 0]))

        print("done ", file_name.split('/')[-1])

    return


### Crown Heights

In [ ]:
geojson_ch ={
    "type": "Feature",
    "properties": {},
    "geometry": {
    "type": "Polygon",
    "coordinates": [
          [
            [
              -73.96373748779297,
              40.68063802521456
            ],
            [
              -73.96064758300781,
              40.64977817702306
            ],
            [
              -73.9215087890625,
              40.651601518462535
            ],
            [
              -73.91876220703125,
              40.678815477435386
            ],
            [
              -73.96373748779297,
              40.68063802521456
            ]
          ]
    ]
    }
}

In [ ]:
geojson_bounds_ch = geojson_ch.get('bbox', calculate_bounds(geojson_ch))
geojson_bounds_ch
bounds_ch = transform_bounds(CRS.from_epsg(4326), src.crs, *geojson_bounds_ch)
left, bottom, right, top = bounds_ch
CrownHeights = rasterio.windows.from_bounds(left, bottom, right, top, src.transform).round_offsets()

In [ ]:
# Use 87
def save_and_process_subsets_CH(file_name):
    t0 = time.time()
    with rasterio.open(file_name) as src:
        out_meta = src.meta

        t1 = time.time()

        # blue green red
        bands = src.read([1,2,3], window=CrownHeights)

        # Create RGB natural color composite
        rgb = np.dstack((bands[2], bands[1], bands[0]))

        t2 = time.time()

        first_step = t1 - t0
        second_step = t2 - t1

        #print(f"first step:{first_step}")
        #print(f"second step:{second_step}")

        profile = src.profile.copy()

        #  Set dtype to int16, and specify LZW compression.
        profile.update(
            driver = "GTiff",
            dtype = rasterio.int16,
            count = 3,
            compress ='lzw',
            height = rgb.shape[0],
            width = rgb.shape[1],
            transform =rasterio.windows.transform(CrownHeights, src.transform),
            photometric ="RGB")

        with rasterio.open('/Users/mansi/Downloads/CrownHeights/' + file_name.split('/')[-1] , 'w', **profile) as dst:
            dst.write(np.moveaxis(rgb.astype(rasterio.int16), [0, 1, 2], [2, 1, 0]))


        print("done ", file_name.split('/')[-1])

    return


### Gowanus Canal

In [ ]:
geojson_gc ={
    "type": "Feature",
    "properties": {},
    "geometry": {
    "type": "Polygon",
    "coordinates": [
          [
            [
              -74.02055740356445,
              40.67139458989753
            ],
            [
              -74.01300430297852,
              40.65199222800328
            ],
            [
              -73.97747039794922,
              40.67829474034605
            ],
            [
              -73.98845672607422,
              40.68532434783304
            ],
            [
              -74.02055740356445,
              40.67139458989753
            ]
          ]
    ]
    }
}


In [ ]:
geojson_gc2 ={
    "type": "Feature",
    "properties": {},
    "geometry": {
    "type": "Polygon",
    "coordinates": [
          [
            [
              -74.0287971496582,
              40.6749098501149
            ],
            [
              -74.01609420776367,
              40.648085029646715
            ],
            [
              -73.96459579467773,
              40.68154928041783
            ],
            [
              -73.9870834350586,
              40.6920928987952
            ],
            [
              -74.0287971496582,
              40.6749098501149
            ]
          ]
    ]
    }
}

In [ ]:
geojson_bounds_gc = geojson_gc2.get('bbox', calculate_bounds(geojson_gc2))
geojson_bounds_gc
bounds_gc = transform_bounds(CRS.from_epsg(4326), src.crs, *geojson_bounds_gc)
left, bottom, right, top = bounds_gc
GowanusCanal = rasterio.windows.from_bounds(left, bottom, right, top, src.transform).round_offsets()

In [ ]:
# Use 87
def save_and_process_subsets_GC(file_name):
    t0 = time.time()
    with rasterio.open(file_name) as src:
        out_meta = src.meta

        t1 = time.time()

        # blue green red
        bands = src.read([1,2,3], window=GowanusCanal)

        # Create RGB natural color composite
        rgb = np.dstack((bands[2], bands[1], bands[0]))

        t2 = time.time()

        first_step = t1 - t0
        second_step = t2 - t1

        #print(f"first step:{first_step}")
        #print(f"second step:{second_step}")

        profile = src.profile.copy()

        #  Set dtype to int16, and specify LZW compression.
        profile.update(
            driver = "GTiff",
            dtype = rasterio.int16,
            count = 3,
            compress ='lzw',
            height = rgb.shape[0],
            width = rgb.shape[1],
            transform =rasterio.windows.transform(GowanusCanal, src.transform),
            photometric ="RGB")

        with rasterio.open('/Users/mansi/Downloads/GowanusCanal2/' + file_name.split('/')[-1] , 'w', **profile) as dst:
            dst.write(np.moveaxis(rgb.astype(rasterio.int16), [0, 1, 2], [2, 1, 0]))

        print("done ", file_name.split('/')[-1])
    return


### Prospect Park

In [ ]:
geojson_pp ={
    "type": "Feature",
    "properties": {},
    "geometry": {
    "type": "Polygon",
    "coordinates": [
          [
            [
              -73.97918701171875,
              40.67660231671296
            ],
            [
              -73.98519515991211,
              40.676081562335845
            ],
            [
              -73.98502349853516,
              40.639358127326865
            ],
            [
              -73.95069122314453,
              40.64079098062354
            ],
            [
              -73.95034790039062,
              40.676992879826386
            ],
            [
              -73.97918701171875,
              40.67660231671296
            ]

          ]
    ]
    }
}


In [ ]:
geojson_bounds_pp = geojson_pp.get('bbox', calculate_bounds(geojson_pp))
geojson_bounds_pp
bounds_pp = transform_bounds(CRS.from_epsg(4326), src.crs, *geojson_bounds_pp)
left, bottom, right, top = bounds_pp
ProspectPark = rasterio.windows.from_bounds(left, bottom, right, top, src.transform).round_offsets()

In [ ]:
# Use 87
def save_and_process_subsets_PP(file_name):
    t0 = time.time()
    with rasterio.open(file_name) as src:
        out_meta = src.meta

        t1 = time.time()

        # blue green red
        bands = src.read([1,2,3], window=ProspectPark)

        # Create RGB natural color composite
        rgb = np.dstack((bands[2], bands[1], bands[0]))

        t2 = time.time()

        first_step = t1 - t0
        second_step = t2 - t1

        #print(f"first step:{first_step}")
        #print(f"second step:{second_step}")

        profile = src.profile.copy()

        #  Set dtype to int16, and specify LZW compression.
        profile.update(
            driver = "GTiff",
            dtype = rasterio.int16,
            count = 3,
            compress ='lzw',
            height = rgb.shape[0],
            width = rgb.shape[1],
            transform =rasterio.windows.transform(ProspectPark, src.transform),
            photometric ="RGB")

        with rasterio.open('/Users/mansi/Downloads/ProspectPark/' + file_name.split('/')[-1] , 'w', **profile) as dst:
            dst.write(np.moveaxis(rgb.astype(rasterio.int16), [0, 1, 2], [2, 1, 0]))

        print("done ", file_name.split('/')[-1])
    return


### Navy Yards/Exhibition Area

In [ ]:
geojson_ny ={
    "type": "Feature",
    "properties": {},
    "geometry": {
    "type": "Polygon",
    "coordinates": [
          [
            [
              -74.00527954101562,
              40.69144210646147
            ],
            [
              -73.98622512817383,
              40.67282675532094
            ],
            [
              -73.94365310668945,
              40.691051628010236
            ],
            [
              -73.96648406982422,
              40.709141393000124
            ],
            [
              -73.98124694824219,
              40.70862089287528
            ],
            [
              -73.99515151977538,
              40.706799110400006
            ],
            [
              -74.00527954101562,
              40.69144210646147
            ]
          ]
    ]
    }
}


In [ ]:
geojson_bounds_ny = geojson_ny.get('bbox', calculate_bounds(geojson_ny))
geojson_bounds_ny
bounds_ny = transform_bounds(CRS.from_epsg(4326), src.crs, *geojson_bounds_ny)
left, bottom, right, top = bounds_ny
NavyYards = rasterio.windows.from_bounds(left, bottom, right, top, src.transform).round_offsets()

In [ ]:
# Use 87
def save_and_process_subsets_NY(file_name):
    t0 = time.time()
    with rasterio.open(file_name) as src:
        out_meta = src.meta

        t1 = time.time()

        # blue green red
        bands = src.read([1,2,3], window=NavyYards)

        # Create RGB natural color composite
        rgb = np.dstack((bands[2], bands[1], bands[0]))

        t2 = time.time()

        first_step = t1 - t0
        second_step = t2 - t1

        #print(f"first step:{first_step}")
        #print(f"second step:{second_step}")

        profile = src.profile.copy()

        #  Set dtype to int16, and specify LZW compression.
        profile.update(
            driver = "GTiff",
            dtype = rasterio.int16,
            count = 3,
            compress ='lzw',
            height = rgb.shape[0],
            width = rgb.shape[1],
            transform =rasterio.windows.transform(NavyYards, src.transform),
            photometric ="RGB")

        with rasterio.open('/Users/mansi/Downloads/NavyYards/' + file_name.split('/')[-1] , 'w', **profile) as dst:
            dst.write(np.moveaxis(rgb.astype(rasterio.int16), [0, 1, 2], [2, 1, 0]))

        print("done ", file_name.split('/')[-1])
    return


In [ ]:
# TODO don't normalize the bands here, do it in adobe after effects to get better color
# TODO resample the pixels from 3m to 1.5 m, and target greater than 1920 by 1080. For urban areas, biliear resampling. For nature areas, cubic resampling
# TODO in adobe after effects, set the frame rate in the program, not on export

### Final Processing

In [ ]:
# Note: change the save_and_process_subsets variable below to match the AOI you're interested in
def get_all_save_and_process(filename_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        return executor.map(save_and_process_subsets_NY, filename_list)

In [ ]:
# Script for CP (88)
t0 = time.time()
with rasterio.Env(GS_SECRET_ACCESS_KEY=gs_secret_access_key, GS_ACCESS_KEY_ID=gs_access_key):
    client = storage.Client()
    bucket = client.bucket('dxd_project_2022')
    filename_list = []
    for blob in bucket.list_blobs(prefix='UTM-24000/18N/24E-188N/PF-SR'):
        file_end = blob.name.split('/')[-1]
        filename = 'gs://dxd_project_2022/' + blob.name
        filename_list.append(filename)

    get_all_save_and_process(filename_list)

In [ ]:
# Script for Other Areas (87)
t0 = time.time()
with rasterio.Env(GS_SECRET_ACCESS_KEY=gs_secret_access_key, GS_ACCESS_KEY_ID=gs_access_key):
    client = storage.Client()
    bucket = client.bucket('dxd_project_2022')
    filename_list = []
    for blob in bucket.list_blobs(prefix='UTM-24000/18N/24E-187N/PF-SR'):
        file_end = blob.name.split('/')[-1]
        filename = 'gs://dxd_project_2022/' + blob.name
        filename_list.append(filename)

    get_all_save_and_process(filename_list)

done  2018-01-01.tif
done  2018-01-10.tif
done  2018-01-12.tif
done  2018-01-08.tif
done  2018-01-03.tif
done  2018-01-07.tif
done  2018-01-13.tif
done  2018-01-11.tif
done  2018-01-02.tif
done  2018-01-14.tif
done  2018-01-06.tif
done  2018-01-15.tif
done  2018-01-04.tif
done  2018-01-05.tif
done  2018-01-09.tif
done  2018-01-16.tif
done  2018-01-19.tif
done  2018-01-17.tif
done  2018-01-18.tif
done  2018-01-22.tif
done  2018-01-24.tif
done  2018-01-20.tif
done  2018-01-23.tif
done  2018-01-25.tif
done  2018-01-28.tif
done  2018-01-30.tif
done  2018-01-27.tif
done  2018-01-29.tif
done  2018-02-01.tif
done  2018-02-02.tif
done  2018-02-03.tif
done  2018-01-26.tif
done  2018-02-05.tif
done done  2018-02-07.tif
 2018-02-04.tif
done  2018-02-06.tif
done  2018-02-08.tif
done  2018-01-21.tif
done  2018-02-11.tif
done  2018-02-09.tif
done  2018-02-10.tif
done  2018-02-12.tif
done  2018-02-14.tif
done  2018-02-18.tif
done  2018-02-21.tif
done  2018-02-15.tif
done done done  2018-02-20.tif
 20

done  2019-01-31.tif
done  2019-01-28.tif
done  2019-01-30.tif
done  2019-02-02.tif
done done  2019-01-22.tif
 2019-02-01.tif
done  2019-01-29.tif
done done  2019-02-04.tif
 2019-02-07.tif
done done  2019-02-08.tif
 2019-02-06.tif
done  2019-02-05.tif
done  2019-02-09.tif
done  2019-02-13.tif
done  2019-02-11.tif
done  2019-02-10.tif
done  2019-02-03.tif
done  2019-02-17.tif
done  2019-02-16.tif
done  2019-02-14.tif
done  2019-02-18.tif
done  2019-02-20.tif
done  2019-02-12.tif
done  2019-02-15.tif
done  2019-02-19.tif
done  2019-02-22.tif
done  2019-02-23.tif
done  2019-02-21.tif
done  2019-02-25.tif
done  2019-02-24.tif
done  2019-02-26.tif
done  2019-03-01.tif
done  2019-03-05.tif
done  2019-02-28.tif
done  2019-03-03.tif
done  2019-03-02.tif
done  2019-03-04.tif
done  2019-03-06.tif
done  2019-02-27.tif
done  2019-03-11.tif
done  2019-03-09.tif
done  2019-03-08.tif
done done  2019-03-14.tif
 2019-03-12.tif
done done   2019-03-13.tif
2019-03-10.tifdone 
 2019-03-07.tif
done  2019-03

done  2020-02-22.tif
done  2020-02-21.tif
done  2020-02-25.tif
done  2020-02-24.tif
done  2020-03-02.tif
done  2020-03-05.tif
done  2020-02-27.tif
done  2020-03-04.tif
done  2020-03-03.tif
done  2020-02-28.tif
done  2020-02-29.tif
done done  2020-02-26.tif
 2020-03-01.tif
done  2020-03-07.tif
done  2020-03-06.tif
done  2020-03-14.tif
done  2020-03-12.tif
done  2020-03-08.tif
done  2020-03-10.tif
done  2020-03-13.tif
done  2020-03-16.tif
done  2020-03-09.tif
done  2020-03-18.tif
done  2020-03-11.tif
done  2020-03-17.tif
done  2020-03-20.tif
done  2020-03-25.tif
done  2020-03-22.tif
done  2020-03-15.tif
done  2020-03-21.tif
done  2020-03-23.tif
done  2020-03-19.tif
done  2020-03-27.tif
done  2020-03-28.tif
done  2020-03-26.tif
done  2020-03-24.tif
done  2020-03-30.tif
done  2020-03-29.tif
done  2020-03-31.tif
done done   2020-04-01.tif
2020-04-06.tif
done  2020-04-04.tif
done  2020-04-02.tif
done  2020-04-03.tif
done  2020-04-08.tif
done  2020-04-07.tif
done  2020-04-12.tif
done  2020-04

done  2021-03-22.tif
done  2021-03-19.tif
done  2021-03-20.tif
done  2021-03-23.tif
done  2021-03-21.tif
done  2021-03-24.tif
done  2021-03-26.tif
done  2021-03-29.tif
done  2021-03-25.tif
done  2021-04-02.tif
done  2021-03-27.tif
done  2021-03-28.tif
done  2021-03-31.tif
done  2021-04-01.tif
done  2021-04-05.tif
done  2021-04-03.tif
done  2021-04-06.tif
done  2021-04-04.tif
done  2021-04-09.tif
done  2021-03-30.tif
done  2021-04-11.tif
done  2021-04-10.tif
done  2021-04-08.tif
done done  2021-04-07.tif
 2021-04-13.tif
done  2021-04-14.tif
done  2021-04-15.tif
done  2021-04-12.tif
done  2021-04-18.tif
done  2021-04-16.tif
done  2021-04-17.tif
done  2021-04-20.tif
done  2021-04-26.tif
done  2021-04-24.tif
done  2021-04-21.tif
done  2021-04-22.tif
done  2021-04-27.tif
done  2021-04-29.tif
done  2021-04-28.tif
done  2021-04-25.tif
done  2021-04-23.tif
done  2021-04-30.tif
done  2021-04-19.tif
done  2021-05-01.tif
done  2021-05-02.tif
done done  2021-05-07.tif
 2021-05-05.tif
done done  20

In [ ]:
rasterio.__version__

'1.2.10'

In [ ]:
rasterio.__gdal_version__

'3.4.1'

In [ ]:
osr.GetPROJVersionMajor()

NameError: name 'osr' is not defined

In [ ]:
osr.GetPROJVersionMinor()

NameError: name 'osr' is not defined

Extra:

In [ ]:
def save_file(rgb, unique_file_name):
    # Write an array as a raster
    # For the new file's profile, we start with the profile of the source
    profile = src.profile.copy()

    #  Set dtype to int16, and specify LZW compression.
    profile.update(
        driver = "GTiff",
        dtype = rasterio.int16,
        count = 3,
        compress ='lzw',
        transform =rasterio.windows.transform(Window1, src.transform),
        height = rgb.shape[0],
        width = rgb.shape[1],
        photometric ="RGB")

    with rasterio.open('/Users/mansi/Downloads/CentralPark/' + unique_file_name , 'w', **profile) as dst:
        dst.write(np.moveaxis(rgb.astype(rasterio.int16), [0, 1, 2], [2, 1, 0]))

In [ ]:
!echo "foo"

foo


In [ ]:
!echo "$GDAL_DATA"

/opt/anaconda3/share/gdal


In [ ]:
!echo "$PROJ_LIB"

/opt/anaconda3/share/proj



     active environment : DxD_new3
    active env location : /opt/anaconda3/envs/DxD_new3
            shell level : 2
       user config file : /Users/mansi/.condarc
 populated config files : /Users/mansi/.condarc
          conda version : 4.11.0
    conda-build version : 3.21.4
         python version : 3.8.8.final.0
       virtual packages : __osx=10.16=0
                          __unix=0=0
                          __archspec=1=x86_64
       base environment : /opt/anaconda3  (writable)
      conda av data dir : /opt/anaconda3/etc/conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/osx-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/osx-64
                          https://repo.anaconda.com/pkgs/r/noarch
          package cache : /opt/anaconda3/pkgs
                          /Users/mansi/.conda/pkgs
       envs directories : /opt/anaconda3/envs
        

In [ ]:
!export GDAL_DATA="${CONDA_PREFIX}/share/gdal"

In [ ]:
!export PROJ_LIB="${CONDA_PREFIX}/share/proj"